In [1]:
import pandas as pd
import numpy as np

### Download the data and load it to Pandas. 

You can find them [here](https://drive.google.com/file/d/1NY6cmF9Shjw-dD7BD6bNmfcIVz-kQcFR/view?usp=sharing).

In [2]:
cast = pd.read_csv('data/cast.csv', index_col=None)
cast.head()

,title,year,name,type,character,n
0,Closet Monster,2015,Buffy #1,actor,Buffy 4,NaN
1,Suuri illusioni,1985,Homo $,actor,Guests,22.0
2,Battle of the Sexes,2017,$hutter,actor,Bobby Riggs Fan,10.0
3,Secret in Their Eyes,2015,$hutter,actor,2002 Dodger Fan,NaN
4,Steve Jobs,2015,$hutter,actor,1988 Opera House Patron,NaN


In [3]:
release_dates = pd.read_csv('data/release_dates.csv', index_col=None,
                                      parse_dates=['date'], infer_datetime_format=True)
release_dates.head()

,title,year,country,date
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15
1,#Beings,2015,Romania,2015-01-29
2,#Declimax,2018,Netherlands,2018-01-21
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21
4,#Horror,2015,USA,2015-11-20


### Count the number of movies with "Christmas" in their title for each month, that is released are the USA.

### Count the number of movies with "Christmas" in their title for each month, that are released in Canada.

### Count the number of movies in which the titles start with "The Hobbit" for each month, that are released in the USA.

### Count the number of movies with "Romance" in their title for **each day of the week**, that are released in the USA.

### Count the number of movies with "Action" in their title for **each day of the week**, that are released in the USA.

### On which date was each Judi Dench movie from the 1990s released in the USA?

### In which months do films with the actress Judi Dench tend to be released in the USA?

### In which months do films with the actor Tom Cruise tend to be released in the USA?